# Рекомендация тарифов

В вашем распоряжении данные о поведении клиентов, которые уже перешли на эти тарифы (из проекта курса «Статистический анализ данных»). Нужно построить модель для задачи классификации, которая выберет подходящий тариф. Предобработка данных не понадобится — вы её уже сделали.

Постройте модель с максимально большим значением *accuracy*. Чтобы сдать проект успешно, нужно довести долю правильных ответов по крайней мере до 0.75. Проверьте *accuracy* на тестовой выборке самостоятельно.

###### Цель исследования:
Построить наиболее качественную модель, которая выберет подходящий тариф для клиента оператора мобильной связи «Мегалайн».

###### Ход исследования:
1. Открыть и изучить предоставленные данные и подготовить их к исследованию при необходимости.
2. Разделить исходные данные на обучающую, валидационную и тестовую выборки.
3. Исследовать качество разных моделей, меняя гиперпараметры.
4. Проверить качество модели на тестовой выборке.
5. Проверить модель на вменяемость.
6. Сделать вывод.

## Откройте и изучите файл

Импортируем библиотеки.

In [1]:
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.tree import DecisionTreeClassifier
from sklearn.ensemble import RandomForestClassifier
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import accuracy_score
from sklearn.dummy import DummyClassifier

Откроем файл с данными и изучим их.

In [2]:
df = pd.read_csv('/datasets/users_behavior.csv')
df.head(20)

,calls,minutes,messages,mb_used,is_ultra
0,40.0,311.90,83.0,19915.42,0
1,85.0,516.75,56.0,22696.96,0
2,77.0,467.66,86.0,21060.45,0
3,106.0,745.53,81.0,8437.39,1
4,66.0,418.74,1.0,14502.75,0
5,58.0,344.56,21.0,15823.37,0
6,57.0,431.64,20.0,3738.90,1
7,15.0,132.40,6.0,21911.60,0
8,7.0,43.39,3.0,2538.67,1
9,90.0,665.41,38.0,17358.61,0


In [3]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 3214 entries, 0 to 3213
Data columns (total 5 columns):
 #   Column    Non-Null Count  Dtype  
---  ------    --------------  -----  
 0   calls     3214 non-null   float64
 1   minutes   3214 non-null   float64
 2   messages  3214 non-null   float64
 3   mb_used   3214 non-null   float64
 4   is_ultra  3214 non-null   int64  
dtypes: float64(4), int64(1)
memory usage: 125.7 KB


Файл действительно содержит данные, заявленные в документах к нему.
На всякий случай проверим данные на пропуски и явные дубликаты, несмотря на то, что заявлено о их полной готовности к исследованию. Изменения типов данных не требуются.

In [4]:
df.isna().sum()

calls       0
minutes     0
messages    0
mb_used     0
is_ultra    0
dtype: int64

In [5]:
df.duplicated().sum()

0

Пропусков и дубликатов нет.

Вывод: данные изучены. в предобработке дополнительно не нуждаются. Таблица содержит все необходимые данные, которые и были заявлены в документах. Можно приступать к работе над основной задачей - построению модели по рекоммендации тарифа пользователям.

## Разбейте данные на выборки

1. Объявим две переменные - признаки (features) и целевой признак (target). Целевым признаком будет являться колонка 	is_ultra, остальные колонки таблицы - это features

In [6]:
features = df.drop(['is_ultra'], axis=1)
target = df['is_ultra']

In [7]:
print(features.shape)
print(target.shape)

(3214, 4)
(3214,)


Итак данные разделены на признаки и целевой признак. Далее нужно разделить все данные на обучающую, валидационную и тестовую выборки. Данные должны быть разбиты в соотношении 3:1:1 соответственно.

2. Выделим обучающую выборку. Она должна составить 60% от всего массива данных. Для начала выделим его.

In [8]:
features_train, features_test, target_train, target_test = train_test_split(features, target, test_size=0.4, random_state=12345) 

Размер обучающей выборки получился такой:

In [9]:
print(features_train.shape)
print(target_train.shape)

(1928, 4)
(1928,)


3. Оставшиеся 40% данных разделим пополам на валидационную и тестовую выборки.

In [18]:
features_valid, features_test, target_valid, target_test = train_test_split(features_test, 
                                                                            target_test, 
                                                                            test_size=0.5, 
                                                                            random_state=12345
                                                                           )

Размер валидационной выборки:

In [11]:
print(features_valid.shape)
print(target_valid.shape)

(643, 4)
(643,)


Размер тестовой выборки:

In [12]:
print(features_test.shape)
print(target_test.shape)

(643, 4)
(643,)


Вывод: данные разделены на обучающую, валидационную и тестовую выборки в соотношении 3:1:1

## Исследуйте модели

Так как задача на классификацию (задача модели выбрать для аббонента один из двух возможных тарифов - Смарт(0) или Ультра(1)), для ее решения будем использовать соответствующие модели:
1. Решающее дерево(DecisionTreeClassifier)
2. Случайный лес(RandomForestClassifier)
3. Логистическая регрессия(LogisticRegression)

Для проверки качества моделей будем использовать метрику accuracy. Чем больше получится данный показатель, тем более качественна модель.

### Модель 1. Решающее дерево(DecisionTreeClassifier)

Напишем код, который позволит нам выбрать наиболее качественную из возможных вариантов модели Решающего дерева. Для этого будем перебирать гиперпараметр max_depth от 1 до 10

In [13]:
best_model_tree = None
best_depth_tree = 0
best_result_tree = 0
for depth in range(1, 11):
    model_tree = DecisionTreeClassifier(random_state=12345, max_depth = depth)
    model_tree.fit(features_train, target_train)
    predictions_valid_tree = model_tree.predict(features_valid)
    result_tree = accuracy_score(target_valid, predictions_valid_tree)
    if result_tree > best_result_tree:
        best_model_tree = model_tree
        best_depth_tree = depth
        best_result_tree = result_tree
        
print("Accuracy лучшей модели:", best_result_tree)
print ("Глубина лучшей модели:", best_depth_tree)

Accuracy лучшей модели: 0.7853810264385692
Глубина лучшей модели: 3


In [22]:
best_model_tree = None
best_depth_tree = 0
best_split_tree = 0
best_result_tree = 0
for depth in range(1, 11):
    for split in range(2, 5):
        model_tree = DecisionTreeClassifier(random_state=12345, max_depth = depth, min_samples_split = split)
        model_tree.fit(features_train, target_train)
        predictions_valid_tree = model_tree.predict(features_valid)
        result_tree = accuracy_score(target_valid, predictions_valid_tree)
        if result_tree > best_result_tree:
            best_model_tree = model_tree
            best_depth_tree = depth
            best_split_tree = split
            best_result_tree = result_tree
        
print("Accuracy лучшей модели:", best_result_tree)
print ("Глубина лучшей модели:", best_depth_tree)
print ("Минимальное количество примеров для разделения лучшей модели:", best_split_tree)

Accuracy лучшей модели: 0.8193146417445483
Глубина лучшей модели: 7
Vинимальное количество примеров для разделения лучшей модели: 2


Вывод: Качество (accuracy) лучшей модели Решающего дерева из исследуемых десяти равно 0.7853810264385692 при гиперпараметре max_depth равном 3.

### Модель 2. Случайный лес(RandomForestClassifier)

Аналогично исследуем модель Случайный лес. В данном случае нужно будет найти лучший параметр accuracy, перебирая не только глубину, но и количество деревьев. Для исследования возьмем глубину от 1 до 10, и количество деревьев от 1 до 20.

In [14]:
best_model_forest = None
best_est_forest = 0
best_depth_forest = 0
best_result_forest = 0
for est in range(1, 21):
    for depth in range(1, 11):
        model_forest = RandomForestClassifier(random_state=12345, n_estimators=est, max_depth = depth) 
        model_forest.fit(features_train, target_train)
        predictions_valid_forest = model_forest.predict(features_valid)
        result_forest = accuracy_score(target_valid, predictions_valid_forest)
        if result_forest > best_result_forest:
            best_model_forest = model_forest
            best_est_forest = est
            best_depth_forest = depth
            best_result_forest = result_forest
        
print("Accuracy лучшей модели:", best_result_forest)
print("Количество деревьев лучшей модели:", best_est_forest)
print ("Глубина лучшей модели:", best_depth_forest)

Accuracy лучшей модели: 0.8040435458786936
Количество деревьев лучшей модели: 12
Глубина лучшей модели: 6


In [23]:
best_model_forest = None
best_est_forest = 0
best_depth_forest = 0
best_split_forest = 0
best_result_forest = 0
for est in range(1, 21):
    for depth in range(1, 11):
        for split in range(2, 5):
            model_forest = RandomForestClassifier(random_state=12345, n_estimators=est, max_depth = depth, 
                                                  min_samples_split = split) 
            model_forest.fit(features_train, target_train)
            predictions_valid_forest = model_forest.predict(features_valid)
            result_forest = accuracy_score(target_valid, predictions_valid_forest)
            if result_forest > best_result_forest:
                best_model_forest = model_forest
                best_est_forest = est
                best_depth_forest = depth
                best_split_forest = split
                best_result_forest = result_forest
        
print("Accuracy лучшей модели:", best_result_forest)
print("Количество деревьев лучшей модели:", best_est_forest)
print ("Глубина лучшей модели:", best_depth_forest)
print ("Минимальное количество примеров для разделения лучшей модели:", best_split_forest)

Accuracy лучшей модели: 0.8348909657320872
Количество деревьев лучшей модели: 6
Глубина лучшей модели: 8
Минимальное количество примеров для разделения лучшей модели: 3


Вывод: Качество (accuracy) лучшей модели Случайного леса из исследуемых равно 0.8040435458786936 при гиперпараметре max_depth равном 6, и количесве деревьев равном 12.

### Модель 3. Логистическая регрессия(LogisticRegression)

Так же исследуем третью модель - Логистическую регрессию. Добавим гиперпараметры: solver='lbfgs' и max_iter=1000

In [15]:
model_regression = LogisticRegression(random_state=12345, solver='lbfgs', max_iter=1000)
model_regression.fit(features_train, target_train)
predictions_valid_regression = model_regression.predict(features_valid)
result_regression = accuracy_score(target_valid, predictions_valid_regression)

print("Accuracy LogisticRegression:", result_regression)

Accuracy LogisticRegression: 0.7107309486780715


Вывод: Accuracy модели LogisticRegression равно 0.7107309486780715

### Вывод. 

Наиболее качественной себя показала модель Случайного леса(RandomForestClassifier).Ее показатель accuracy составил 0.8040435458786936 при гиперпараметре max_depth равном 6, и количесве деревьев равном 12.

На втором месте по качеству оказалась модель Решающее дерево(DecisionTreeClassifier) с показателем accuracy 0.7853810264385692.

Хуже всех по качеству оказалась модель Логистическая регрессия(LogisticRegression) с показателем accuracy 0.7107309486780715.

## Проверьте модель на тестовой выборке

Наиболее качественной себя показала модель Случайного леса(RandomForestClassifier). Проверим данную модель на тестовой выборке.

In [16]:
model_forest = RandomForestClassifier(random_state=12345, n_estimators = 12, max_depth = 6)
model_forest.fit(features_train, target_train)
predictions_test_forest = model_forest.predict(features_test)
accuracy_test_forest = accuracy_score(target_test, predictions_test_forest)

print("Accuracy модели случайного леса на тестовой выборке:", accuracy_test_forest)

Accuracy модели случайного леса на тестовой выборке: 0.7947122861586314


### Вывод. 
Accuracy модели случайного леса на тестовой выборке составила 0.7947122861586314, что выше заданного обязательного параметра 0.75. Таким образом, делаем вывод, что выбранная модель работает успешно.

## (бонус) Проверьте модели на адекватность

С помощью DummyClassifier создадим простейшую модель, которая будет предсказывать наиболее часто встречающийся класс. Найдем показатель accuracy для этой модели и сравним его с показателем accuracy для модели случайного леса, который показал свою эффективность в тестовой выборке в предыдущем задании. Если они будут различаться, значит модель адекватна, и обучилась верно, она действительно делает предсказание на основе всех параметров, и это не случайные предсказания.

In [17]:
dummy_model = DummyClassifier(strategy='most_frequent', random_state=12345)
dummy_model.fit(features_train, target_train)
predictions_dummy = dummy_model.predict(features_test)
print("Accuracy модели DummyClassifier:", accuracy_score(target_test, predictions_dummy))

Accuracy модели DummyClassifier: 0.6842923794712286


Вывод: Модель Случайного леса, которую мы опрелили как качественную и успешную, адекватна, т.к. показатель Accuracy нашей модели составил 0.7947122861586314, что выше показателя Accuracy простейшей модели DummyClassifier (0.6842923794712286). Это значит, что наша модель делает не случайные предсказания.
Кроме того, мы также можем сделать вывод, что каждая из построенных нами моделей в предыдущем задании, адекватна.

## Общий вывод

#### Этап 1. Изучение данных.
В ходе этого этапа был открыт файл и изучены данные. Они действительно соответствовали документации. Так же была проведена проверка на пропуски и дубликаты. Их обнаружено не было, предобработка данных не потребовалась.

#### Этап 2. Разделение исходных данных на обучающую, валидационную и тестовую выборки.
Данные были разделены на три выборки в соотношении 3:1:1. 60% данных было отведено на обучающую выборку, 20% - на валидационную и еще 20% на тестовую.

#### Этап 3. Исследование качества разных моделей.
В ходе работы были исследованы 3 разных модели:
1. Решающее дерево(DecisionTreeClassifier)
2. Случайный лес(RandomForestClassifier)
3. Логистическая регрессия(LogisticRegression)
Для проверки качества моделей была использована метрика accuracy.

Наиболее качественной себя показала модель Случайного леса(RandomForestClassifier).Ее показатель accuracy составил 0.8040435458786936 при гиперпараметре max_depth равном 6, и количесве деревьев равном 12.
На втором месте по качеству оказалась модель Решающее дерево(DecisionTreeClassifier) с показателем accuracy 0.7853810264385692.
Хуже всех по качеству оказалась модель Логистическая регрессия(LogisticRegression) с показателем accuracy 0.7107309486780715.

#### Этап 4. Проверка качества модели на тестовой выборке.
На тестовой выборке была проверена модель Случайного леса(RandomForestClassifier), которая показала наиболее высокий показатель качества. Accuracy модели случайного леса на тестовой выборке составила 0.7947122861586314, что всего на 0.01 меньше, чем на валидационной. Модель рабочая.

#### Этап 5. Проверка модели на адекватность.
Была создана простейшая модель, которая предсказывает наиболее часто встречающийся класс. В результате сравнения качества простейшей модели и выбранной нами эффективной модели, было выявлено, что показатель Accuracy нашей модели составил 0.7947122861586314, это выше показателя Accuracy простейшей модели DummyClassifier (0.6842923794712286). Это значит, что наша модель делает не случайные предсказания.

## Чек-лист готовности проекта

Поставьте 'x' в выполненных пунктах. Далее нажмите Shift+Enter.

- [x] Jupyter Notebook открыт
- [x] Весь код исполняется без ошибок
- [x] Ячейки с кодом расположены в порядке исполнения
- [x] Выполнено задание 1: данные загружены и изучены
- [x] Выполнено задание 2: данные разбиты на три выборки
- [x] Выполнено задание 3: проведено исследование моделей
    - [x] Рассмотрено больше одной модели
    - [x] Рассмотрено хотя бы 3 значения гипепараметров для какой-нибудь модели
    - [x] Написаны выводы по результатам исследования
- [x] Выполнено задание 3: Проведено тестирование
- [x] Удалось достичь accuracy не меньше 0.75
